# Air Pollution and the COVID-19 Pandemic
The COVID-19 pandemic has caused an unprecedented disruption in every aspect of human activity, on a global scale. Billions of people have been affected by the disease in numerous ways, and felt its impact in their daily lives. Fortunately, the pandemic had a few beneficial effects as well. The various measures implemented by governments all over the world, like travel restrictions, border closures and lockdowns, have led to the decrease of air pollution in many countries. Air pollution is itself a major health issue, causing millions of deaths on a yearly basis. In this notebook we are going to explore the decrease of air pollution during the COVID-19 pandemic in a number of countries and regions. 

## Nitrogen Dioxide 
Nitrogen dioxide (NO<sub>2</sub>) is one of the major air pollutants associated with health issues. The majority of nitrogen dioxide in urban areas, is emitted from the burning of fossil fuels, therefore cities with heavy car traffic have high concentration of the gas. It is also produced from power plants and other industrial activity. Continued exposure to NO<sub>2</sub> can cause various health problems, such as inflammation of the lungs, bronchitis, and other respiratory illnesses.

## Sentinel-5p Satellite
Our analysis is based on data provided by Copernicus, the earth observation programme coordinated by the European Union, and implemented by various organizations such as the European Space Agency (ESA). The Sentinel-5P satellite is part of the Copernicus programme, and is focused on air quality and air pollution. It monitors various traces gases such as CO, NO<sub>2</sub>, SO<sub>2</sub>, O<sub>3</sub>, CH<sub>4</sub> and others. We are going to access and analyze the NO<sub>2</sub> data from Sentinel-5P for our analysis

## Air Pollution in Italy
Italy is one of the european countries that have been signficantly affected by the COVID-19 pandemic. As of June 2020, there have been more than 230.000 confirmed cases, and about 33.000 thousands deaths attributed to the coronavirus. The italian government implemente various measures to contain the pandemic, and eventually imposed a nationwide lockdown on March 9 2020.

![Italy Pollution](images/pollution_Italy.png)

As we can see, nitrogen dioxide concentration was significantly decreased in Italy during March 2020, compared to the same period last year. This can be at least partly attributed to the COVID-19 measures implemented by the italian government, and the reduction of traffic and industrial activity.

## Air Pollution in Lombardy
The Lombardy region of Italy was severely affected by the COVID-19 pandemic, having one of the highest rates of confirmed cases in the country. Lombardy and its capital city Milan, is heavily polluted due to the presence of various factories and other industrial activity in the area.

![Lombardy Pollution](images/pollution_Lombardy.png)

As we can see Lombardy was a pollution hotspot in March 2019. The government measures for the COVID-19 pandemic decreased NO<sub>2</sub> concentration significantly.

## Air Pollution in Greece
The greek government implemented various measures to contain the COVID-19 outbreak. On March 22 2020 a nationwide lockdown was imposed. 

![Greece Pollution](images/pollution_Greece.png)

NO<sub>2</sub> concentration was decreased in Greece during March 2020, especially in the regions of Central Macedonia and Attica. 

## Air Pollution in Attica
The Attica region of Greece encompasses Athens, the country's capital. Air quality is a problem in Athens, where NO<sub>2</sub> levels are occasionally above safe levels.

![Attica Pollution](images/pollution_Attica.png)

The measures imposed by the greek government for the COVID-19 pandemic, led to a significant decrease in NO<sub>2</sub> levels in Attica, due to reduced traffic and other activities. 